# **1. Install Dependencies**

In [ ]:
!pip install paho-mqtt joblib scikit-learn matplotlib


# **Fungsi dan penjelasan:**

* paho-mqtt → library yang memungkinkan Python menjadi MQTT Subscriber / Publisher

* joblib → digunakan untuk memuat model ML (.pkl) dari Session 3–4

* scikit-learn → library Machine Learning yang digunakan untuk prediksi

* matplotlib → untuk visualisasi grafik data IoT

> 📌 Ini adalah fondasi agar Python mampu menerima data sensor, menjalankan ML, dan menampilkan grafik.

# **2. Import Libraries**

In [2]:
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import paho.mqtt.client as mqtt


# **Fungsi masing-masing:**

* json — parsing JSON dari ESP32 (karena ESP32 mengirim JSON)

* joblib — load model ML

* numpy — manipulasi numerik

* pandas — logging & membuat tabel dataset baru

* matplotlib — visualisasi

* datetime — mencatat timestamp setiap data masuk

* mqtt — koneksi ke MQTT broker HiveMQ (colab sebagai subscriber)

> 📌 Cell ini menyiapkan semua tools yang digunakan dalam proses IoT + ML.

# **3. Upload Model**

In [3]:
from google.colab import files
uploaded = files.upload()  # browse file
model = joblib.load("iot_temp_model.pkl")
print("Model loaded!")


Saving iot_temp_model.pkl to iot_temp_model.pkl
Model loaded!


# **Fungsi:**

* Meng-upload file model ML yang telah Anda latih sebelumnya (model klasifikasi)

* File model ini dibuat dari dataset temperature + humidity → label

* Setelah diload, model siap melakukan prediksi real-time

> 📌 Tanpa model ini, pipeline IoT → ML tidak dapat berjalan.

# **4. MQTT Config**

In [4]:
MQTT_BROKER = "broker.hivemq.com"
TOPIC_SENSOR = "iot/class/session5/sensor"
TOPIC_OUTPUT = "iot/class/session5/output"

print("MQTT Broker:", MQTT_BROKER)


MQTT Broker: broker.hivemq.com


# **Fungsi dan penjelasan:**

* MQTT_BROKER = server tempat data dari ESP32 dikirim

* TOPIC_SENSOR = topik tempat ESP32 mengirim data ke Colab

* TOPIC_OUTPUT = topik tempat Colab mengirim hasil ML kembali ke ESP32

> 📌 Topik harus sama antara Python ↔ ESP32 agar komunikasi berjalan.

# **5. Callbacks MQTT**

In [6]:
logs = []

def on_connect(client, userdata, flags, rc):
    print("Connected →", rc)
    client.subscribe(TOPIC_SENSOR)

def on_message(client, userdata, msg):
    data = json.loads(msg.payload.decode())
    temp = float(data["temp"])
    hum  = float(data["hum"])
    ts   = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    print(f"[{ts}] ESP32 → Temp={temp}, Hum={hum}")

    # ML Prediction
    X = [[temp, hum]]
    pred = model.predict(X)[0]
    print("ML Prediction →", pred)

    # logging
    logs.append([ts, temp, hum, pred])

    # SEND BACK to ESP32
    if pred == "Panas":
        client.publish(TOPIC_OUTPUT, "ALERT_ON")
    else:
        client.publish(TOPIC_OUTPUT, "ALERT_OFF")

    print("Output sent →", "ALERT_ON" if pred=="Panas" else "ALERT_OFF")


**on_connect():**

* Dijalankan saat Colab berhasil terkoneksi ke MQTT broker

* Colab langsung subscribe ke topik sensor ESP32
→ sehingga bisa menerima data real-time

**TUGAS 1 — Menerima data dari ESP32**

    ESP32 mengirim JSON:

      {"temp": 29.5, "hum": 70.0}

**Colab menerima → parsing JSON → ambil nilai:**
    
    temperature

    humidity

**TUGAS 2 — Prediksi ML**

    X = [[temp, hum]]
    pred = model.predict(X)[0]

**Model akan mengeluarkan:**

    “Dingin”

    “Normal”

    “Panas”

Inilah inti integrasi Machine Learning.

**TUGAS 3 — Logging**

    logs.append([ts, temp, hum, pred])

Semua data disimpan sebagai list → nanti jadi CSV → dashboard.

**TUGAS 4 — Kirim output ke ESP32**

    if pred == "Panas":
        client.publish(TOPIC_OUTPUT, "ALERT_ON")
    else:
        client.publish(TOPIC_OUTPUT, "ALERT_OFF")

Jika suhu tinggi, ESP32 akan menyalakan LED/buzzer.

**TUGAS 5 — Menyiapkan feedback loop IoT**

    Ini menjadikan ML sebagai:

    “Otak Pengambil Keputusan” IoT


# **6. Start MQTT Client**

In [7]:
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(MQTT_BROKER, 1883, 60)
client.loop_start()

print("Listening for ESP32 data...")


Listening for ESP32 data...


/tmp/ipython-input-1496475644.py:1: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


# **Fungsi:**

* Membuat Python sebagai MQTT client

* Menjalankan listener asinkron (loop_start)

* Memulai koneksi real-time dengan broker HiveMQ

> 📌 Setelah cell ini, data ESP32 akan langsung masuk.

# **7. Wait & Check Log**

In [ ]:
import time

for i in range(10):   # cek 10 kali
    print("Waiting for data...", i)
    time.sleep(2)


# **Fungsi:**

* Memberi waktu untuk Colab menerima data

* Digunakan untuk test awal saja

# **8. Save Log to CSV**

In [ ]:
df = pd.DataFrame(logs, columns=["timestamp", "temperature", "humidity", "prediction"])
df.to_csv("iot_realtime_predictions.csv", index=False)

print("Saved → iot_realtime_predictions.csv")
df.tail()


# **Fungsi:**

* Mengonversi log menjadi tabel

* Menyimpan dataset baru sebagai CSV

* Dataset ini bisa dipakai lagi untuk retraining model

> 📌 Berperan sebagai feedback loop: Real-time IoT → Dataset.

# **9. Realtime Visualization**

In [ ]:
if len(df) > 0:
    plt.figure(figsize=(10,5))
    plt.plot(df["temperature"], label="Temp (°C)")
    plt.plot(df["humidity"], label="Humidity (%)")
    plt.title("Realtime Sensor Data from ESP32")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("Belum ada data.")


# **Fungsi:**

* Membuat grafik perubahan suhu & kelembapan secara real-time

* Digunakan untuk:

    - memonitor sensor

    - memvalidasi kualitas data

    - mendeteksi anomali visual

> 📌 Sangat penting saat debugging ESP32 atau model ML.

# **10. Manual Test Prediction**

In [19]:
temp = 33.1
hum  = 55.2

pred = model.predict([[temp, hum]])[0]
print("Test Prediction:", pred)


Test Prediction: Panas


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


# **Fungsi:**

* Untuk menguji apakah model ML berjalan normal tanpa data ESP32

* Digunakan saat debugging model

# **🎯 HASIL AKHIR NOTEBOOK**

**Notebook ini membentuk pipeline:**

| Stage       | Fungsi                           |
| ----------- | -------------------------------- |
| ESP32       | Kirim data sensor                |
| MQTT        | Media komunikasi                 |
| Colab       | Menerima data + jalankan ML      |
| ML Model    | Memprediksi kondisi suhu         |
| Output MQTT | Mengirim perintah balik ke ESP32 |
| ESP32       | Menyalakan LED/Buzzer            |
